In [2]:
import pandas as pd
import numpy as np

In [5]:
file_to_open = 'https://raw.githubusercontent.com/quibes/burnout_GT_hack/main/Additional%20file%205.%20Raw%20Dataset%20N%3D973.csv'
dataset = pd.read_csv(file_to_open, sep=';')
dataset.head()


,AGE,GENDER,MARITAL STATUS,HOUSEHOLD,RESIDENCE TYPE,REGION,SMOKING,SPECIALTY,CLASSIFICATION,WORKPLACE,...,Burnout Q6,Burnout Q7,Burnout Q8,Burnout Q9,Burnout Q10,Job Satisfaction Q1,Job Satisfaction Q2,Job Satisfaction Q3,Job Satisfaction Q4,Job Satisfaction Q5
0,42,Male,Married,4-6 Persons,Owned Falt/House,"Central (Amman, Zarqa, Madaba, Al-Balqaa)",Non-smoker,Medical,Consultant,Jordanian Royal Medical Services,...,Sometimes,Rarely,Rarely,Rarely,Rarely,Agree,Agree,Disagree,Agree,Disagree
1,30,Male,Married,1-3 Persons,Owned Falt/House,"North (Irbid, Ajloun, Jerash, Mafraq)",Smoker,Medical,Resident,Jordanian Royal Medical Services,...,Rarely,Sometimes,Never,Sometimes,Very Often,Strongly Agree,Agree,Disagree,Agree,Disagree
2,38,Male,Married,1-3 Persons,Rented Falt/House,"Central (Amman, Zarqa, Madaba, Al-Balqaa)",Smoker,Medical,Specialist,Jordanian Royal Medical Services,...,Sometimes,Almost Never,Almost Never,Sometimes,Sometimes,Strongly disagree,Neutral/Undecided,Agree,Disagree,Agree
3,37,Male,Married,4-6 Persons,Rented Falt/House,"North (Irbid, Ajloun, Jerash, Mafraq)",Non-smoker,Medical,Specialist,Jordanian Royal Medical Services,...,Rarely,Rarely,Never,Rarely,Never,Strongly Agree,Strongly Agree,Strongly disagree,Strongly agree,Strongly disagree
4,39,Male,Married,4-6 Persons,Owned Falt/House,"North (Irbid, Ajloun, Jerash, Mafraq)",Smoker,Surgical,Specialist,Jordanian Royal Medical Services,...,Always,Sometimes,Very Often,Often,Always,Strongly disagree,Strongly disagree,Neutral/Undecided,Strongly disagree,Strongly agree


In [6]:
for column in dataset.columns[27:32]:
  dataset[column] = dataset[column].str.strip().str.lower()

dataset.replace({'Never': 0, 'Almost Never': 1, 'Rarely': 2, 'Sometimes': 3, 'Often': 4, 'Very Often': 5, 'Always': 6}, inplace = True)
dataset[dataset.columns[[27, 28, 30]]] = dataset[dataset.columns[[27, 28, 30]]].replace({'strongly disagree': 0, 'disagree': 1, 'neutral/undecided': 2, 'agree': 3, 'strongly agree': 4})
dataset[dataset.columns[[29, 31]]] = dataset[dataset.columns[[29, 31]]].replace({'strongly disagree': 4, 'disagree': 3, 'neutral/undecided': 2, 'agree': 1, 'strongly agree': 0})

dataset.replace({'1-3 Persons': 0, '4-6 Persons': 1, '>6 Persons': 2, 'Male': 0, 'Female': 1, 'Married': 0, 'Single': 1, 'Owned Falt/House': 0, 'Rented Falt/House': 1, 
                 'Non-smoker': 0, 'Smoker': 1, 'Medical ': 0, 'Medical': 0, 'Surgical': 1, 'Full-Time': 0, 'Part-Time': 1, '<700 JDs': 0, 
                 '700-1400 JDs': 1, '>1400 JDs': 2, '<40': 0, '40-48': 1, '>48': 2, '1-3 on-calls': 1, '>3': 2, 'No': 0, 'Yes': 1}, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split 

dataset_dum = pd.get_dummies(dataset, drop_first=True).drop(columns=['Burnout Q1', 'Burnout Q2', 'Burnout Q3', 'Burnout Q4', 'Burnout Q5', 'Burnout Q6', 'Burnout Q7', 'Burnout Q8', 'Burnout Q9', 'Burnout Q10'])

dataset_dum.drop(columns=['Job Satisfaction Q1', 'Job Satisfaction Q2',
       'Job Satisfaction Q3', 'Job Satisfaction Q4', 'Job Satisfaction Q5'], inplace=True)

data_dum_target = dataset_dum['burnout_index']

X_train, X_test, y_train, y_test = train_test_split(dataset_dum.drop(columns=['burnout_index']), data_dum_target, test_size=0.3)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV

model = LinearRegression()
model.fit(X_train, y_train)

pd.DataFrame({'columns': X_train.columns, 'coefs': np.around(model.coef_, decimals=4)})

y_preds = model.predict(X_test)

mean_squared_error(y_test, y_preds)

r2_score(y_test, model.predict(X_test))